CSCI E-599 Management of DynamoDB database called `demographics`
=====


Dave Dowey
===


---


![xkcd_penguins](https://imgs.xkcd.com/comics/march_of_the_penguins.png)


<a id='back-to-top'></a>

### [*Creation of the database*](#creation)
- [Creation of the dynamodb table called `demographics`](#create_table)
- [Batch populate the table with `batch_write_item`](#batch_from_web)   
- [Populate the table from JSON](#populate_from_json)  



### [*Manage the database*](#management)
- [Inquire about the table](#describe_table)
- [Update with new attributes](#update_attributes)
- [Put item into the table](#put_item)
- [Query the total number of items](#query_number_items)
- [Delete the table](#delete_table)


### [*Metadata table for `demographics`*](#metadata)
- [Creation of the metadata table called `demographics_meta`](#create_meta)
- [Populate the table with new metadata items](#populate_meta)   
- [Query the attributes](#query_meta)
- [Delete the metadata table](#delete_meta)


-------

### Preliminaries - libraries to load

In [41]:
import os
import sys
os.environ["TZ"]="UTC"
import time
import boto3

from datetime import datetime
import bs4 as bs
import json
import decimal
import glob

import numpy as np
import pandas as pd
from pprint import pprint

In [2]:
## Markdown CSS
from IPython.core.display import HTML
HTML("""
<style>

div.cell { 
    margin-top:1em;
    margin-bottom:1em;
}

div.text_cell_render h1 {
    font-size: 1.8em;
    line-height:1.2em;
    text-align:center;
}

div.text_cell_render h2 {
margin-bottom: -0.2em;
}

table tbody tr td:first-child, 
table tbody tr th:first-child, 
table thead tr th:first-child, 
table tbody tr td:nth-child(4), 
table thead tr th:nth-child(4) {
    background-color: #edf4e8;
}

div.text_cell_render { 
    font-family: 'Garamond';
    font-size:1.4em;
    line-height:1.3em;
    padding-left:3em;
    padding-right:3em;
}

div#notebook-container    { width: 95%; }
div#menubar-container     { width: 65%; }
div#maintoolbar-container { width: 99%; }

</style>
""")

[back-to-top](#back-to-top)
<a id='creation'></a>

*Creation of the database*  
=====


<a id='create_table'></a>


Create the dynamodb table called `demographics`
----


In [24]:
dynamodb_client = boto3.client('dynamodb', region_name='us-east-1')

try:
    table = dynamodb_client.create_table(
        TableName='demographics',
        KeySchema=[
            {
                'AttributeName': 'pmcid', 
                'KeyType': 'HASH'
            },
            {
                'AttributeName': 'pmid',
                'KeyType': 'RANGE' 
            }
        ], 
        AttributeDefinitions=[
            {
                'AttributeName': 'pmcid', 
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'pmid',
                'AttributeType': 'S'
            },

        ], 
        ProvisionedThroughput={
            'ReadCapacityUnits': 10, 
            'WriteCapacityUnits': 10
        }
    )

    dynamodb_client.get_waiter('table_exists').wait(TableName='demographics')
    print("Table status:",  table['TableDescription']['TableStatus'])
    print("Item count:", table['TableDescription']['ItemCount'])
    
except dynamodb_client.exceptions.ResourceInUseException:
    print("Table in use error - do you really want to recreate the table?")
    pass
except OSError as err:
    print("OS error: {0}".format(err))
except ValueError:
    print("Could not convert data to an integer.")
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Table status: CREATING
Item count: 0


[back-to-top](#back-to-top)
<a id='batch_from_web'></a>


Batch populate the table with `batch_write_item`
------

----


### `batch_write_item` from the XML files in a folder

In [ ]:
item_count = 0
itemset_array = []
now = datetime.now().strftime("%Y-%m-%d")
file_path = '/home/dave/datapubmed/targetarticles/'

client = boto3.client('dynamodb')

file_list = glob.glob(file_path + 'PMC*.nxml')
#choice = np.random.randint(10000, size=1000)
for f in range(len(file_list)): 
#for f in choice: 
    item_count += 1
    tree = read_xml(file_list[f])
    if tree is not None:
        dict_article_meta = parse_article_meta(tree)
        pmid = dict_article_meta['pmid'];# print (pmid)
        if (pmid == ''):
            pmid = 'PMID missing'
        pmc = dict_article_meta['pmc']; #print (pmc)
        
        tree_title = tree.find('//title-group/article-title')
        if tree_title is not None:
            title = [t for t in tree_title.itertext()]
            sub_title = tree.xpath('//title-group/subtitle/text()')
            title.extend(sub_title)
            title = [t.replace('\n', ' ').replace('\t', ' ') for t in title]
            if len(title):
                full_title = ' '.join(title)
            else:
                full_title = 'Title missing'
        else:
            full_title = 'Title missing'
            
        item_dict = {
            "PutRequest": {
                "Item": {
                    "pmcid": {"S": pmc},
                    "pmid": {"S": pmid},
                    "date_processed": {"S": now},
                    "title": {"S": full_title},
                }
            }
        }

        itemset_array.append(item_dict)
        
    if (item_count % 25 == 0):
        response = client.batch_write_item(RequestItems={ "demographics": itemset_array})
        itemset_array = []
    #if (item_count % 10000 == 0):
    #    print ("The item_count is: %s " % item_count)

### Create a JSON file with the same `batch_write_item` input

In [ ]:
item_count = 0
itemset_array = []
now = datetime.now().strftime("%Y-%m-%d")

file_path = '/home/dave/datapubmed/targetarticles/'

client = boto3.client('dynamodb')

file_list = glob.glob(file_path + 'PMC*.nxml')

for f in range(len(file_list)): 

    tree = read_xml(file_list[f])
    if tree is not None:
        dict_article_meta = parse_article_meta(tree)
        pmid = dict_article_meta['pmid'];# print (pmid)
        if pmid == '':
            pmid = 'PMID missing'
        pmc = dict_article_meta['pmc']; #print (pmc)
        
        tree_title = tree.find('//title-group/article-title')
        if tree_title is not None:
            title = [t for t in tree_title.itertext()]
            sub_title = tree.xpath('//title-group/subtitle/text()')
            title.extend(sub_title)
            title = [t.replace('\n', ' ').replace('\t', ' ') for t in title]
            if len(title):
                full_title = ' '.join(title)
            else:
                full_title = 'Title missing'
        else:
            full_title = 'Title missing'
            
        item_dict = {
            "PutRequest": {
                "Item": {
                    "pmcid": {"S": pmc},
                    "pmid": {"S": pmid},
                    "date_processed": {"S": now},
                    "title": {"S": full_title},
                }
            }
        }

        itemset_array.append(item_dict)
output_dict = { "demographics": itemset_array}

### Dump the JSON file to disk

In [ ]:
jsondata = json.dumps(output_dict)
f = open("output_dict.json","w")
f.write(jsondata)
f.close()

### Create a JSON file with the list of PMCIDs in the demographics JSON file created above

In [ ]:
#print([d['PutRequest']['Item']['pmcid']['S']  for d in output_dict['demographics']]) 

PMCIDs_in_demographics = [d['PutRequest']['Item']['pmcid']['S']  for d in output_dict['demographics']]
jsondata = json.dumps(pmcid_in_demographics)
f = open("PMCIDs_in_demographics.json","w")
f.write(jsondata)
f.close()

### Check the distribution of the contents of the `output_dict` dict

In [ ]:
pprint(Counter( [d['PutRequest']['Item']['pmcid']['S']  for d in output_dict['demographics']]).most_common(40))
pprint(Counter( [d['PutRequest']['Item']['pmid']['S']  for d in output_dict['demographics']]).most_common(40))
pprint(Counter( [d['PutRequest']['Item']['date_processed']['S']  for d in output_dict['demographics']]).most_common(40))
pprint(Counter( [d['PutRequest']['Item']['title']['S']  for d in output_dict['demographics']]).most_common(40))

[back-to-top](#back-to-top)
<a id='populate_from_json'></a>


Populate the table from JSON
------


----


### Fix the missing PMIDs in the table - these are missing in the XML files

In [ ]:
openaccess_df = pd.read_csv('oa_file_list.csv')
print(openaccess_df.shape)

openaccess_df['PMCID'] = openaccess_df['Accession ID'].str[3:]
print(openaccess_df.count(axis=0))
openaccess_df.head()

In [ ]:
demographics_in_oa = openaccess_df[openaccess_df['PMCID'].isin(PMCIDs_in_demographics)]
print(len(demographics_in_oa))
print(demographics_in_oa[:10])
demographics_in_oa = demographics_in_oa.set_index('PMCID')
demographics_in_oa.head()

In [ ]:
missing_pmid = []
for d in output_dict['demographics']:
    mykey = d['PutRequest']['Item']['pmcid']['S']
    if (d['PutRequest']['Item']['pmid']['S'] == ''):
        missing_pmid.append(mykey)
        d['PutRequest']['Item']['pmid']['S'] = str(int(demographics_in_oa['PMID'].loc[mykey]))
        
print(len(missing_pmid))
print(missing_pmid[:10])

In [ ]:
for mykey in missing_pmid:
    print(int(demographics_in_oa['PMID'].loc[mykey]))

In [ ]:
resource = boto3.resource('dynamodb')
table = resource.Table('demographics')

for i in range(len(missing_pmid)):
    mykey = missing_pmid[i]

    table.update_item(
        Key={'pmcid': mykey},
        UpdateExpression="set pmid = :s",
        ExpressionAttributeValues={
            ':s': str(int(demographics_in_oa['PMID'].loc[mykey]))
        },
    )

### Create a JSON file with the table items from the `output_dict.json` file 

In [27]:
# In PutRequest style
demographic_json = []
for d in output_dict['demographics']:
    itemdict = {
        "pmcid": {"S": str(d['PutRequest']['Item']['pmid']['S'])},
        "pmid":  {"S": str(d['PutRequest']['Item']['pmcid']['S'])},
        "date_processed":  {"S": str(d['PutRequest']['Item']['date_processed']['S'])},
        "title":  {"S": unidecode(d['PutRequest']['Item']['title']['S'])}
        }
    demographic_json.append(itemdict)

In [ ]:
# In normal put style
now = datetime.now().strftime("%Y-%m-%d")
demographic_json = []
for d in output_dict['demographics']:
    itemdict = {
        "pmcid": str(d['PutRequest']['Item']['pmcid']['S']),
        "pmid":  str(d['PutRequest']['Item']['pmid']['S']),
        "date_processed":  str(now),
        "title":  unidecode(d['PutRequest']['Item']['title']['S'])
        }
    demographic_json.append(itemdict)

In [ ]:
jsondata = json.dumps(demographic_json)
f = open("demographic_json.json","w")
f.write(jsondata)
f.close()

#f = open('demographic_json.json')
#request_items = json.loads(f.read())

### Batch put from the JSON object

#### *This is what has populated the current table*

In [ ]:
dynamodb_resource = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb_resource.Table('demographics')

with table.batch_writer() as batch:
    for i in range(len(demographic_json)):
        batch.put_item(
            Item=demographic_json[i]
        )
        if (i % 1000 == 0):
            print ("The item_count is: %s " % (i))

### Normal  put from the JSON object

In [ ]:
dynamodb_resource = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb_resource.Table('demographics')

for i in range(len(demographic_json)):
    table.put_item(
        Item=demographic_json[i]
    )
#    if (i % 10 == 0):
#        print ("The item_count is: %s " % (i))

### Normal  put from a JSON file

In [ ]:
dynamodb_resource = boto3.resource('dynamodb', region_name='us-east-1')
input_file = "demographic_json.json"

try:
    table = dynamodb_resource.Table('demographics')
    print("Instantiate a table: ",table.creation_date_time)
    print("Ready to load data\n")
    incr = 0
    with open(input_file) as json_file:
        itemset = json.load(json_file, parse_float = decimal.Decimal)
        for item in itemset:
            incr += 1
            #pmcid = item['pmcid']
            #title = item['title']
            #date_processed = item['date_processed']
            #print("Adding record # ", incr," pmcid: ",pmcid," title: ",title,)
            #if (incr % 10000 == 0):
            #    print("Adding record # ", incr)
            table.put_item(
               Item={
                   'pmcid': item,
                   #'pmcid': pmcid,
                   #'title': title,
                   #'date_processed': date_processed,
                }
            )
            
except dynamodb_client.exceptions.ResourceNotFoundException:
    print("Table does not exist - cannot delete it")
    pass            
except OSError as err:
    print("OS error: {0}".format(err))
except ValueError:
    print("Could not convert data to an integer.")
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

[back-to-top](#back-to-top)
<a id='management'></a>

*Manage the database*  
=====

<a id='describe_table'></a>

Inquire about the table
------

----


In [36]:
dynamodb_client = boto3.client('dynamodb')

try:
    response = dynamodb_client.describe_table(TableName='demographics')
    pprint(response)
    
except dynamodb_client.exceptions.ResourceNotFoundException:
    print("Table does not exist - cannot describe it")
    pass

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '572',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Mon, 09 Apr 2018 20:29:51 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '3345078066',
                                      'x-amzn-requestid': '0LT9U92U7PT3G947US2P38GG2VVV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': '0LT9U92U7PT3G947US2P38GG2VVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0},
 'Table': {'AttributeDefinitions': [{'AttributeName': 'pmcid',
                                     'AttributeType': 'S'}],
           'CreationDateTime': datetime.datetime(2018, 4, 7, 19, 9, 36, 304000, tzinfo=tzlocal()),
           'ItemCount': 294019,
           'KeySchema': [{'Attribu

In [37]:
dynamodb_resource = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb_resource.Table('demographics')

def get_table_metadata(table_name):
    """
    Get some metadata about chosen table.
    """
    table = dynamodb_resource.Table(table_name)

    return {
        'num_items': table.item_count,
        'primary_key_name': table.key_schema[0],
        'status': table.table_status,
        'bytes_size': table.table_size_bytes,
        'global_secondary_indices': table.global_secondary_indexes
    }

demographics_metata = get_table_metadata('demographics')
pprint(demographics_metata)

{'bytes_size': 105658874,
 'global_secondary_indices': None,
 'num_items': 294019,
 'primary_key_name': {'AttributeName': 'pmcid', 'KeyType': 'HASH'},
 'status': 'ACTIVE'}


[back-to-top](#back-to-top)
<a id='update_attributes'></a>


Update with new attributes
------

----


 ### Update an entire dict item from loaded JSON object of input attributes input_json

In [ ]:
# Update from loaded JSON object of input attributes input_json

responses = []
dynamodb_resource = resource('dynamodb', region_name='us-east-1')
table = dynamodb_resource.Table('demographics')

for i in range(len(input_json)):
    #mykey = input_json[i]['pmcid']

    table.update_item(Item=input_json[i])
       
#    if (i % 10000 == 0):
#        print ("The item_count is: %s " % (i))

In [ ]:
# Update one attribute from loaded JSON object of input attributes input_json

responses = []
dynamodb_resource = resource('dynamodb', region_name='us-east-1')
table = dynamodb_resource.Table('demographics')

for i in range(len(input_json)):
    mykey = input_json[i]['pmcid']

    table.update_item(
        Key={'pmcid': mykey},
        UpdateExpression="set pmid = :s",
        ExpressionAttributeValues={
            ':s': input_json[i]['pmid']
        },
    )

    #print ("The item_count is: %s " % (i))

[back-to-top](#back-to-top)
<a id='put_item'></a>

Put item into the table
------

----


In [ ]:
from boto3 import resource
from boto3.dynamodb.conditions import Key

# The boto3 dynamoDB resource
dynamodb_resource = resource('dynamodb', region_name='us-east-1')
table = dynamodb_resource.Table('demographics')

def read_table_item(table_name, pk_name, pk_value):
    """
    Return item read by primary key.
    """
    table = dynamodb_resource.Table(table_name)
    response = table.get_item(Key={pk_name: pk_value})

    return response

def add_item(table_name, col_dict):
    """
    Add one item (row) to table. col_dict is a dictionary {col_name: value}.
    """
    table = dynamodb_resource.Table(table_name)
    response = table.put_item(Item=col_dict)

    return response

def delete_item(table_name, pk_name, pk_value):
    """
    Delete an item (row) in table from its primary key.
    """
    table = dynamodb_resource.Table(table_name)
    response = table.delete_item(Key={pk_name: pk_value})

    return

### Get an item from the table

In [38]:
dynamodb_resource = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb_resource.Table('demographics')
pmcidd_list = ['2064904']

for i in range(len(pmcidd_list)):

    mykey = pmcidd_list[i]
    response = table.get_item(Key={'pmcid': mykey})
    pprint(response)

{'Item': {'date_processed': '2018-04-07',
          'pmcid': '2064904',
          'pmid': '17883851',
          'sentences': [{'section': 'Sample size',
                         'text': 'A sample of 100 subjects in each group would '
                                 'demonstrate an increase in this "mainly '
                                 'breastfeeding" rate from 46% (control group) '
                                 'to 67% (intervention group) with alpha '
                                 '[confidence level] = 0.05 and power of 80%.'},
                        {'section': 'Background',
                         'text': 'At the fifty-fifth World Health Assembly, '
                                 'the World Health Organization (WHO) '
                                 'recommended that optimal infant nutrition '
                                 'was exclusive breastfeeding for the first '
                                 'six months of life, followed by the '
                         

[back-to-top](#back-to-top)
<a id='query_number_items'></a>


Query the total number of items
----


In [40]:
dynamodb_resource = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb_resource.Table('demographics')

start_time = time.time()

response = table.scan()
pmcids_in_demographics = list(map(lambda d: d['pmcid'], response['Items']))

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    pmcids_in_demographics.extend(list(map(lambda d: d['pmcid'], response['Items'])))
    

elapsed_time = (time.time() - start_time) / 60.

In [39]:
print(len(pmcids_in_demographics))
print (elapsed_time)

print(pmcids_in_demographics[:20])

294019
2.0219762563705443
['2977546', '5651857', '5504832', '4310883', '4906605', '5743172', '5405840', '4785788', '4763193', '3921260', '3554429', '4210062', '4093809', '1785373', '3893583', '3514223', '4061537', '5102273', '5837366', '3809218']


[back-to-top](#back-to-top)
<a id='delete_table'></a>


Delete the `demographics` table
----


In [26]:
dynamodb_client = boto3.client('dynamodb', region_name='us-east-1')

try:
    dynamodb_client.delete_table(TableName='demographics_alt')
    print('Deleted table')
        
except dynamodb_client.exceptions.ResourceNotFoundException:
    print("Table does not exist - cannot delete it")
    pass
except OSError as err:
    print("OS error: {0}".format(err))
except ValueError:
    print("Could not convert data to an integer.")
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Deleted table


[back-to-top](#back-to-top)
<a id='metadata'></a>

*Metadata table  `demographics_meta`*
=====


<a id='create_meta'></a>


Creation of the metadata table called `demographics_meta`
----

-----

In [42]:
dynamodb_client = boto3.client('dynamodb', region_name='us-east-1')

try:
    table = dynamodb_client.create_table(
        TableName='demographics_meta',
        KeySchema=[
            {
                'AttributeName': 'source', 
                'KeyType': 'HASH'
            }
        ], 
        AttributeDefinitions=[
            {
                'AttributeName': 'source', 
                'AttributeType': 'S'
            },
        ], 
        ProvisionedThroughput={
            'ReadCapacityUnits': 5, 
            'WriteCapacityUnits': 5
        }
    )

    dynamodb_client.get_waiter('table_exists').wait(TableName='demographics_meta')
    print("Table status:",  table['TableDescription']['TableStatus'])
    print("Item count:", table['TableDescription']['ItemCount'])
    
except dynamodb_client.exceptions.ResourceInUseException:
    print("Table in use error - do you really want to recreate the table?")
    pass
except OSError as err:
    print("OS error: {0}".format(err))
except ValueError:
    print("Could not convert data to an integer.")
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Table status: CREATING
Item count: 0


[back-to-top](#back-to-top)
<a id='populate_meta'></a>


Populate the table with new metadata items
------

----


## Populate from a list in memory

In [60]:
dynamodb_client = boto3.client('dynamodb', region_name='us-east-1')
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
small_list_of_ids = pmcids_in_demographics[:20]

response = dynamodb_client.put_item(
    Item={
       'source': {"S": "demographics"},
       'pmcids':{"NS": small_list_of_ids},
       'items': {"N": str(len(small_list_of_ids))},
       'date_updated': {"S": now} ,
    },
    TableName='demographics_meta',
)

## Populate from a list in a JSON file

In [61]:
jsondata = json.dumps(small_list_of_ids)
f = open("small_list_of_ids.json","w")
f.write(jsondata)
f.close()

In [62]:
dynamodb_client = boto3.client('dynamodb', region_name='us-east-1')
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#file_path = '/home/dave/datapubmed/'
#input_file = file_path + "inS3BucketIDs.json"
input_file = "small_list_of_ids.json"
try:
    with open(input_file) as json_file:
        itemset = json.load(json_file)
        response = dynamodb_client.put_item(
            Item={
               'source': {"S": "saved_json"},
               'pmcids':{"NS": itemset},
               'items': {"N": str(len(itemset))},
               'date_updated': {"S": now} ,
            },
            TableName='demographics_meta',
        )
            
except OSError as err:
    print("OS error: {0}".format(err))
except ValueError:
    print("Could not convert data to an integer.")
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise            

[back-to-top](#back-to-top)
<a id='query_meta'></a>


Query the metadata table on attributes
------

----


In [64]:
dynamodb_resource = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb_resource.Table('demographics_meta')
item_list = ['demographics']

for i in range(len(item_list)):

    mykey = item_list[i]
    response = table.get_item(Key={'source': mykey})
    pprint(response)

{'Item': {'date_updated': '2018-04-09 21:36:15',
          'items': Decimal('20'),
          'pmcids': {Decimal('1785373'),
                     Decimal('2977546'),
                     Decimal('3514223'),
                     Decimal('3554429'),
                     Decimal('3809218'),
                     Decimal('3893583'),
                     Decimal('3921260'),
                     Decimal('4061537'),
                     Decimal('4093809'),
                     Decimal('4210062'),
                     Decimal('4310883'),
                     Decimal('4763193'),
                     Decimal('4785788'),
                     Decimal('4906605'),
                     Decimal('5102273'),
                     Decimal('5405840'),
                     Decimal('5504832'),
                     Decimal('5651857'),
                     Decimal('5743172'),
                     Decimal('5837366')},
          'source': 'demographics'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-a

[back-to-top](#back-to-top)
<a id='delete_meta'></a>


Delete the metadata table
----


In [26]:
dynamodb_client = boto3.client('dynamodb', region_name='us-east-1')

try:
    dynamodb_client.delete_table(TableName='demographics_meta')
    print('Deleted table')
        
except dynamodb_client.exceptions.ResourceNotFoundException:
    print("Table does not exist - cannot delete it")
    pass
except OSError as err:
    print("OS error: {0}".format(err))
except ValueError:
    print("Could not convert data to an integer.")
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Deleted table


In [ ]:
%save createtabledemographics.py

In [ ]:
%run createtabledemographics.py